# Тестовые задачи по взаимодействию с SparkAPI

Установка спарка на машину

In [ ]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=0f00005e67c2d820a1fa81c48b4d2aacd36e49857276a2c85c0616ae34ed9657
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


Обратимся к гиту, где лежат некоторые файлы для выполнения задач.

In [ ]:
!git clone https://github.com/databricks/LearningSparkV2.git

Cloning into 'LearningSparkV2'...
remote: Enumerating objects: 1720, done.
remote: Counting objects: 100% (1720/1720), done.
remote: Compressing objects: 100% (1036/1036), done.
remote: Total 1720 (delta 546), reused 1691 (delta 541), pack-reused 0 (from 0)
Receiving objects: 100% (1720/1720), 76.97 MiB | 8.78 MiB/s, done.
Resolving deltas: 100% (546/546), done.
Updating files: 100% (768/768), done.


# Задачи

## 1. Возможности спарка

In [ ]:
#Собираем библиотеки
import requests
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import StringType, DateType, FloatType, IntegerType
from google.colab import files
import pandas as pd
from google.colab import files

#ссылка на загрузку CSV файла
url = f'https://drive.google.com/uc?id=1ZuLZDoPboHnM1P8m3PflIe70zUGvjsRq'
#Загружаем файл
response = requests.get(url)
#Проверяем статус ответа, если ок, то открываем файл car в памяти
if response.status_code == 200:
    with open('car.csv', 'wb') as f:
        f.write(response.content)
    print("Файл загружен")
else:
    print(f"Файл не загрузился {response.status_code}")

Файл загружен


In [ ]:
#Делаем функцию, которая принимает загружаемый файл и открывает его как объект спарка

#вытаскиваем имя выгружаемого файла
file_name = str(f).split("name='")[1].split("'")[0]

def create_spark_object(file_name):
    if str(f).lower().find(file_name) > 1:
        spark = SparkSession.builder.appName('Test').getOrCreate()
        dt = spark.read.format("csv").option("header", "true").load(file_name)
        return dt
df = create_spark_object(file_name)

df.show(1, vertical = True)

-RECORD 0-------------------------
 manufacturer_name | Subaru       
 model_name        | Outback      
 transmission      | automatic    
 color             | silver       
 odometer_value    | 190000       
 year_produced     | 2010         
 engine_fuel       | gasoline     
 engine_has_gas    | False        
 engine_type       | gasoline     
 engine_capacity   | 2.5          
 body_type         | universal    
 has_warranty      | False        
 state             | owned        
 drivetrain        | all          
 price_usd         | 10900.0      
 is_exchangeable   | False        
 location_region   | Минская обл. 
 number_of_photos  | 9            
 up_counter        | 13           
 feature_0         | False        
 feature_1         | True         
 feature_2         | True         
 feature_3         | True         
 feature_4         | False        
 feature_5         | True         
 feature_6         | False        
 feature_7         | True         
 feature_8         |

In [ ]:
#Select
df.select("manufacturer_name","model_name").show(1)
df.select(df["manufacturer_name"],df["model_name"]).show(1)
df.select(F.col("manufacturer_name"), F.col("model_name")).show(3)

+-----------------+----------+
|manufacturer_name|model_name|
+-----------------+----------+
|           Subaru|   Outback|
+-----------------+----------+
only showing top 1 row

+-----------------+----------+
|manufacturer_name|model_name|
+-----------------+----------+
|           Subaru|   Outback|
+-----------------+----------+
only showing top 1 row

+-----------------+----------+
|manufacturer_name|model_name|
+-----------------+----------+
|           Subaru|   Outback|
|           Subaru|   Outback|
|           Subaru|  Forester|
+-----------------+----------+
only showing top 3 rows



In [ ]:
#filter
NAME = 'Audi'

df\
   .select("manufacturer_name","model_name","transmission", "color")\
   .filter(F.col("manufacturer_name") == NAME)\
   .filter("transmission = 'automatic' and color = 'black'").show(2)

+-----------------+----------+------------+-----+
|manufacturer_name|model_name|transmission|color|
+-----------------+----------+------------+-----+
|             Audi|        TT|   automatic|black|
|             Audi|        A6|   automatic|black|
+-----------------+----------+------------+-----+
only showing top 2 rows



In [ ]:
#distinct_count
df.select("manufacturer_name").distinct().count()

55

In [ ]:
#count
df.count()

38531

In [ ]:
#GROUPBY() and ORDERBY()
df.groupBy("manufacturer_name").count().orderBy(F.col("count").desc()).show(5)

+-----------------+-----+
|manufacturer_name|count|
+-----------------+-----+
|       Volkswagen| 4243|
|             Opel| 2759|
|              BMW| 2610|
|             Ford| 2566|
|          Renault| 2493|
+-----------------+-----+
only showing top 5 rows



In [ ]:
# Переименовываем колонку
df = df.withColumnRenamed("manufacturer_name", "manufacturer")

In [ ]:
#Создаем новую колонку
df = df.withColumn("next_year", F.col("year_produced") - 1)

In [ ]:
#Посмотрим типы колонок
df.printSchema()

root
 |-- manufacturer: string (nullable = true)
 |-- model_name: string (nullable = true)
 |-- transmission: string (nullable = true)
 |-- color: string (nullable = true)
 |-- odometer_value: string (nullable = true)
 |-- year_produced: string (nullable = true)
 |-- engine_fuel: string (nullable = true)
 |-- engine_has_gas: string (nullable = true)
 |-- engine_type: string (nullable = true)
 |-- engine_capacity: string (nullable = true)
 |-- body_type: string (nullable = true)
 |-- has_warranty: string (nullable = true)
 |-- state: string (nullable = true)
 |-- drivetrain: string (nullable = true)
 |-- price_usd: string (nullable = true)
 |-- is_exchangeable: string (nullable = true)
 |-- location_region: string (nullable = true)
 |-- number_of_photos: string (nullable = true)
 |-- up_counter: string (nullable = true)
 |-- feature_0: string (nullable = true)
 |-- feature_1: string (nullable = true)
 |-- feature_2: string (nullable = true)
 |-- feature_3: string (nullable = true)
 |-- 

In [ ]:
#посмотрим метрики по чиловым столбцам
df.select('odometer_value', 'year_produced', 'engine_capacity', 'price_usd', 'number_of_photos', 'up_counter', 'duration_listed').describe().show()

+-------+------------------+------------------+-----------------+-----------------+-----------------+------------------+------------------+
|summary|    odometer_value|     year_produced|  engine_capacity|        price_usd| number_of_photos|        up_counter|   duration_listed|
+-------+------------------+------------------+-----------------+-----------------+-----------------+------------------+------------------+
|  count|             38531|             38531|            38521|            38531|            38531|             38531|             38531|
|   mean| 248864.6384469648|2002.9437336170874|2.055161106928777|6639.971021255605|9.649061794399314|16.306091199294073|  80.5772494874257|
| stddev|136072.37652978086| 8.065730511309935|0.671177667208744|6428.152018202911|6.093216996872852| 43.28693309422311|112.82656864261321|
|    min|                 0|              1942|              0.2|              1.0|                1|                 1|                 0|
|    max|           

In [ ]:
#изменим тыпы колонок
df.withColumn("odometer_value", df["odometer_value"].cast(IntegerType()))\
  .withColumn("year_produced", df["year_produced"].cast(IntegerType()))\
  .withColumn("engine_capacity", df["engine_capacity"].cast(FloatType()))\
  .withColumn("price_usd", df["price_usd"].cast(FloatType()))\
  .withColumn("number_of_photos", df["number_of_photos"].cast(IntegerType()))\
  .withColumn("up_counter", df["up_counter"].cast(IntegerType()))\
  .withColumn("duration_listed", df["duration_listed"].cast(IntegerType()))\
  .withColumn("next_year", df["next_year"].cast(IntegerType()))\
  .printSchema()

root
 |-- manufacturer: string (nullable = true)
 |-- model_name: string (nullable = true)
 |-- transmission: string (nullable = true)
 |-- color: string (nullable = true)
 |-- odometer_value: integer (nullable = true)
 |-- year_produced: integer (nullable = true)
 |-- engine_fuel: string (nullable = true)
 |-- engine_has_gas: string (nullable = true)
 |-- engine_type: string (nullable = true)
 |-- engine_capacity: float (nullable = true)
 |-- body_type: string (nullable = true)
 |-- has_warranty: string (nullable = true)
 |-- state: string (nullable = true)
 |-- drivetrain: string (nullable = true)
 |-- price_usd: float (nullable = true)
 |-- is_exchangeable: string (nullable = true)
 |-- location_region: string (nullable = true)
 |-- number_of_photos: integer (nullable = true)
 |-- up_counter: integer (nullable = true)
 |-- feature_0: string (nullable = true)
 |-- feature_1: string (nullable = true)
 |-- feature_2: string (nullable = true)
 |-- feature_3: string (nullable = true)
 |-

## 2. Перейдем к написанию небольшого пайплана

Задача: Необходимо сделать пайплайн обработки файла cars.csv . Посчитать по каждому производителю (поле manufacturer_name):

*   Количество объявлений
*   Средний год выпуска автомобилей
*   Минимальную цену
*   Максимальную цену

Выгрузить результат в output.csv


In [ ]:
#Напишем фнкцию, которая производит обработку и выгружает данные
def create_output():
    output = (
        df
        .groupBy('manufacturer')
        .agg(
            F.count('manufacturer').alias('Count'),
            F.round(F.avg('year_produced')).cast(IntegerType()).alias('Avarage'),
            F.min(F.col('price_usd').cast(FloatType())).alias('Min_price'),
            F.max(F.col('price_usd').cast(FloatType())).alias('Max_price')
            )
        )
    output.show(5)
    dt = pd.DataFrame(output.take(5), columns=output.columns)
    dt.to_csv('filename.csv', index=False)
    files.download('filename.csv')
    return dt

create_output()

NameError: name 'df' is not defined

In [ ]:
#Остонавливаем сессию спарк
spark.stop()

NameError: name 'spark' is not defined

### 1. Поиск M&Ms

Дан сsv файл, который подтягивается из гугл диска.
Задача посчитать количество M&Ms по цветовой гамме в пачках в разных штатах.

In [ ]:
#Библиотеки
from pyspark.sql import SparkSession
from pyspark.sql.functions import count
import requests

#ссылка на загрузку CSV файла c гугл Диска
url = f'https://drive.google.com/uc?id=1oZTMFL05rLTN-oqBXl4W-rWS-9bpcphg'

#Загружаем файл
response = requests.get(url)
#Проверяем статус ответа, если ок, то открываем файл в памяти
if response.status_code == 200:
    with open('mnm.csv', 'wb') as f:
        f.write(response.content)
    print("Файл загружен")
else:
    print(f"Файл не загрузился {response.status_code}")

Файл загружен


In [ ]:
#выводим название файла
file_name = str(f).split("name='")[1].split("'")[0]
#вызываем функцию, которая генерит объект объект Spark
df = create_spark_object(file_name)

#посмотрим кол-во конфет в разрезе штатов и цветовой гаммы
count_mnm_state_color = (df.select('State', 'Color', 'Count')
                           .groupBy('State', 'Color')
                           .agg(count("Count").alias("Total"))
                           .orderBy("Total", ascending=False).show())

#проверим конфеты по Калифорнии
count_mnm_NY = (df.select('State','Color','Count')
                  .where(df.State == 'CA')
                  .groupBy('State', 'Color')
                  .agg(count("Count").alias('Total'))
                  .orderBy("Total", ascending=False)
                  .show())
count_mnm_NY

+-----+------+-----+
|State| Color|Total|
+-----+------+-----+
|   CA|Yellow| 1807|
|   WA| Green| 1779|
|   OR|Orange| 1743|
|   TX| Green| 1737|
|   TX|   Red| 1725|
|   CA| Green| 1723|
|   CO|Yellow| 1721|
|   CA| Brown| 1718|
|   CO| Green| 1713|
|   NV|Orange| 1712|
|   TX|Yellow| 1703|
|   NV| Green| 1698|
|   AZ| Brown| 1698|
|   CO|  Blue| 1695|
|   WY| Green| 1695|
|   NM|   Red| 1690|
|   AZ|Orange| 1689|
|   NM|Yellow| 1688|
|   NM| Brown| 1687|
|   UT|Orange| 1684|
+-----+------+-----+
only showing top 20 rows

+-----+------+-----+
|State| Color|Total|
+-----+------+-----+
|   CA|Yellow| 1807|
|   CA| Green| 1723|
|   CA| Brown| 1718|
|   CA|Orange| 1657|
|   CA|   Red| 1656|
|   CA|  Blue| 1603|
+-----+------+-----+



## 3. Поиск слова Spark в тексте.

Дан .md файл с текстом. Задача, выгрузить строки, где есть слово Spark из файла md формата.

In [ ]:
#Выгрузка библиотек
from pyspark.sql import SparkSession
from google.colab import drive
from google.colab import files

#обращаемся к гугл диску
drive.mount('/content/drive', force_remount=True)

#создаем спарк приложение и читаем файл
spark = SparkSession.builder.config('spark.ui.port', '4050').appName('Test').getOrCreate()
file_path = '/content/drive/My Drive/README.md'
strings = spark.read.text(file_path)

#выводим строки с неоьходимым нам словом
filtered = strings.filter(strings.value.contains("Spark"))
filtered.show(100)

Mounted at /content/drive
+--------------------+
|               value|
+--------------------+
|      # Apache Spark|
|Spark is a unifie...|
|rich set of highe...|
|pandas API on Spa...|
|[![PySpark Covera...|
|You can find the ...|
|   ## Building Spark|
|Spark is built us...|
|To build Spark an...|
|["Building Spark"...|
|For general devel...|
|The easiest way t...|
|Spark also comes ...|
|./bin/run-example...|
|MASTER=spark://ho...|
|Testing first req...|
|Spark uses the Ha...|
|Hadoop, you must ...|
|in the online doc...|
|Please review the...|
+--------------------+



In [ ]:
#Останавливаем сессию
spark.stop()

## 4. Создаем схему




### 1. Задание
Необходимо сгенерить данные и создать под них схему.

In [ ]:
# Выгружаем библиотеку
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
# Прописываем схему DDL
schema = "`Id` INT, `First` STRING, `Last` STRING, `Url` STRING, `Published` STRING, `Hits` INT, `Campaigns` ARRAY<STRING>"
# Данные для датафрейма
data = [[1, "Jules", "Damji", "https://tinyurl.1", "1/4/2016", 4535, ["twitter", "LinkedIn"]],
 [2, "Brooke","Wenig", "https://tinyurl.2", "5/5/2018", 8908, ["twitter",
"LinkedIn"]],
 [3, "Denny", "Lee", "https://tinyurl.3", "6/7/2019", 7659, ["web",
"twitter", "FB", "LinkedIn"]],
 [4, "Tathagata", "Das", "https://tinyurl.4", "5/12/2018", 10568,
["twitter", "FB"]],
 [5, "Matei","Zaharia", "https://tinyurl.5", "5/14/2014", 40578, ["web",
"twitter", "FB", "LinkedIn"]],
 [6, "Reynold", "Xin", "https://tinyurl.6", "3/2/2015", 25568,
["twitter", "LinkedIn"]]
 ]
spark = (SparkSession
 .builder
 .appName("Example-3_6")
 .getOrCreate())

# Создаем датафрейм используя данные и схему
blogs_df = spark.createDataFrame(data, schema)

#Выводим данные и структуру схемы
blogs_df.show()
print(blogs_df.schema)
print(' ')
blogs_df.printSchema()

+---+---------+-------+-----------------+---------+-----+--------------------+
| Id|    First|   Last|              Url|Published| Hits|           Campaigns|
+---+---------+-------+-----------------+---------+-----+--------------------+
|  1|    Jules|  Damji|https://tinyurl.1| 1/4/2016| 4535| [twitter, LinkedIn]|
|  2|   Brooke|  Wenig|https://tinyurl.2| 5/5/2018| 8908| [twitter, LinkedIn]|
|  3|    Denny|    Lee|https://tinyurl.3| 6/7/2019| 7659|[web, twitter, FB...|
|  4|Tathagata|    Das|https://tinyurl.4|5/12/2018|10568|       [twitter, FB]|
|  5|    Matei|Zaharia|https://tinyurl.5|5/14/2014|40578|[web, twitter, FB...|
|  6|  Reynold|    Xin|https://tinyurl.6| 3/2/2015|25568| [twitter, LinkedIn]|
+---+---------+-------+-----------------+---------+-----+--------------------+

StructType([StructField('Id', IntegerType(), True), StructField('First', StringType(), True), StructField('Last', StringType(), True), StructField('Url', StringType(), True), StructField('Published', StringTyp

In [ ]:
blogs_df\
       .withColumn('Big_Hints', F.col('Hits') > 5000)\
       .withColumn('Auth_data', (F.concat(F.col('First'), F.col('Last'), F.col('Id'))))\
       .sort(F.col('Hits').desc())\
       .show()

+---+---------+-------+-----------------+---------+-----+--------------------+---------+-------------+
| Id|    First|   Last|              Url|Published| Hits|           Campaigns|Big_Hints|    Auth_data|
+---+---------+-------+-----------------+---------+-----+--------------------+---------+-------------+
|  5|    Matei|Zaharia|https://tinyurl.5|5/14/2014|40578|[web, twitter, FB...|     true|MateiZaharia5|
|  6|  Reynold|    Xin|https://tinyurl.6| 3/2/2015|25568| [twitter, LinkedIn]|     true|  ReynoldXin6|
|  4|Tathagata|    Das|https://tinyurl.4|5/12/2018|10568|       [twitter, FB]|     true|TathagataDas4|
|  2|   Brooke|  Wenig|https://tinyurl.2| 5/5/2018| 8908| [twitter, LinkedIn]|     true| BrookeWenig2|
|  3|    Denny|    Lee|https://tinyurl.3| 6/7/2019| 7659|[web, twitter, FB...|     true|    DennyLee3|
|  1|    Jules|  Damji|https://tinyurl.1| 1/4/2016| 4535| [twitter, LinkedIn]|    false|  JulesDamji1|
+---+---------+-------+-----------------+---------+-----+----------------

### 2. Задание
Необходимо создать схему и положить в нее данные из GitHub. <a name="4.2">

In [ ]:
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.sql import SparkSession

#Моделим схему
file_schema = StructType([StructField('CallNumber', IntegerType(), True),
                          StructField('UnitID', StringType(), True),
                          StructField('InciedentNumber', IntegerType(),True),
                          StructField('CallType', StringType(), True),
                          StructField('CallDate', StringType(), True),
                          StructField('WatchDate', StringType(), True),
                          StructField('CallFinalDisposition', StringType(), True),
                          StructField('AvailableDtTm', StringType(), True),
                          StructField('Address', StringType(), True),
                          StructField('City', StringType(), True),
                          StructField('Zipcode', IntegerType(), True),
                          StructField('Battalion', StringType(), True),
 StructField('StationArea', StringType(), True),
 StructField('Box', StringType(), True),
 StructField('OriginalPriority', StringType(), True),
 StructField('Priority', StringType(), True),
 StructField('FinalPriority', IntegerType(), True),
 StructField('ALSUnit', BooleanType(), True),
 StructField('CallTypeGroup', StringType(), True),
 StructField('NumAlarms', IntegerType(), True),
 StructField('UnitType', StringType(), True),
 StructField('UnitSequenceInCallDispatch', IntegerType(), True),
 StructField('FirePreventionDistrict', StringType(), True),
 StructField('SupervisorDistrict', StringType(), True),
 StructField('Neighborhood', StringType(), True),
 StructField('Location', StringType(), True),
 StructField('RowID', StringType(), True),
 StructField('Delay', FloatType(), True)])

#Читаем датафрейм CSV из GitHub
spark = SparkSession.builder.config('spark.ui.port', '4050').appName('Test').getOrCreate()
str_file = "/content/LearningSparkV2/databricks-datasets/learning-spark-v2/sf-fire/sf-fire-calls.csv"
fire_df = spark.read.csv(str_file, header = True, schema = file_schema)

fire_df.show(5)
fire_df.printSchema()

+----------+------+---------------+----------------+----------+----------+--------------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+-------------+-------+-------------+---------+--------+--------------------------+----------------------+------------------+--------------------+--------------------+-------------+---------+
|CallNumber|UnitID|InciedentNumber|        CallType|  CallDate| WatchDate|CallFinalDisposition|       AvailableDtTm|             Address|City|Zipcode|Battalion|StationArea| Box|OriginalPriority|Priority|FinalPriority|ALSUnit|CallTypeGroup|NumAlarms|UnitType|UnitSequenceInCallDispatch|FirePreventionDistrict|SupervisorDistrict|        Neighborhood|            Location|        RowID|    Delay|
+----------+------+---------------+----------------+----------+----------+--------------------+--------------------+--------------------+----+-------+---------+-----------+----+----------------+--------+---------

## 5. Работа с строками

Создать датафрейм из объекта Row

In [ ]:
from pyspark.sql import Row

#содержимое строки
rows = [Row("Matei Zaharia", "CA"), Row("Reynold Xin", "CA")]
author_row = spark.createDataFrame(rows, ["Authors","Region"])
author_row.show()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.10/dist-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.10/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
cd /content/

## 6. Поиск не пустых значений

Нужно посмотреть причины вызовов пожарных частей - значения должны быть не Null.

Для выпонения надо выполнить [4.2](#4.2)  



In [ ]:
#Читаем датафрейм CSV из GitHub
from pyspark.sql import SparkSession

fire_df.select(F.col("CallType"))\
       .filter(F.col("CallType").isNotNull())\
       .groupBy(F.col("CallType"))\
       .agg(F.count('CallType').alias('Count'))\
       .orderBy(F.col("Count").desc())\
       .show(100,False)

+--------------------------------------------+------+
|CallType                                    |Count |
+--------------------------------------------+------+
|Medical Incident                            |113794|
|Structure Fire                              |23319 |
|Alarms                                      |19406 |
|Traffic Collision                           |7013  |
|Citizen Assist / Service Call               |2524  |
|Other                                       |2166  |
|Outside Fire                                |2094  |
|Vehicle Fire                                |854   |
|Gas Leak (Natural and LP Gases)             |764   |
|Water Rescue                                |755   |
|Odor (Strange / Unknown)                    |490   |
|Electrical Hazard                           |482   |
|Elevator / Escalator Rescue                 |453   |
|Smoke Investigation (Outside)               |391   |
|Fuel Spill                                  |193   |
|HazMat                     

## 7. Замена столбцов и их удаление

Дан датафрейм из Задачи [4.2](#4.2) необходимо изменить тип временных колонок для дальнейшей работы  с ними. Таже необходимо подготовить ответы на вопросы -
* Посмотреть за какие года у были инциденты.
* Какие виды пожара были в 2018 году.
* В какие месяцы 2018 года было больше всего вызовов пожарных?
* В каком районе Сан Франциско произошло больше всего звонков о пожаре в 2018?
* В каких районах было худшее время реагирования пожарных в 2018 году?
* На какой неделе 2018 года было больше всего вызовов пожарных?
* Как использовать тип файлов Паркет, для хранения этих данных.

In [ ]:
#Задача У всех временных столбцов тип поля string
fire_df.select('CallDate','WatchDate','AvailableDtTm').printSchema()

#заменим типы и удалим старые колонки
fire_df = (fire_df
           .withColumn("IncidentDate", F.to_timestamp("CallDate", "MM/dd/yyyy"))
           .drop("CallDate")
           .withColumn("OnWatchDate", F.to_timestamp("WatchDate","MM/dd/yyyy"))
           .drop("WatchDate")
           .withColumn("AvailableDtTS", F.to_timestamp("AvailableDtTm", "MM/dd/yyyy hh:mm:ss a"))
           .drop("AvailableDtTm"))

#Проверка
fire_df.printSchema()

root
 |-- CallDate: string (nullable = true)
 |-- WatchDate: string (nullable = true)
 |-- AvailableDtTm: string (nullable = true)

root
 |-- CallNumber: integer (nullable = true)
 |-- UnitID: string (nullable = true)
 |-- InciedentNumber: integer (nullable = true)
 |-- CallType: string (nullable = true)
 |-- CallFinalDisposition: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Zipcode: integer (nullable = true)
 |-- Battalion: string (nullable = true)
 |-- StationArea: string (nullable = true)
 |-- Box: string (nullable = true)
 |-- OriginalPriority: string (nullable = true)
 |-- Priority: string (nullable = true)
 |-- FinalPriority: integer (nullable = true)
 |-- ALSUnit: boolean (nullable = true)
 |-- CallTypeGroup: string (nullable = true)
 |-- NumAlarms: integer (nullable = true)
 |-- UnitType: string (nullable = true)
 |-- UnitSequenceInCallDispatch: integer (nullable = true)
 |-- FirePreventionDistrict: string (nullable = 

In [ ]:
#Задача 1 за какие года были инциденты
fire_df.select(F.year('IncidentDate').alias('Year')).distinct().orderBy(F.year('IncidentDate')).show(100,False)

+----+
|Year|
+----+
|2000|
|2001|
|2002|
|2003|
|2004|
|2005|
|2006|
|2007|
|2008|
|2009|
|2010|
|2011|
|2012|
|2013|
|2014|
|2015|
|2016|
|2017|
|2018|
+----+



In [ ]:
#Задача 2 какие виды пожаров были в 2018
fire_df.select(F.col("CallType"))\
       .filter(F.year('IncidentDate') == 2018)\
       .groupBy(F.col("CallType"))\
       .agg(F.count('CallType').alias('Count'))\
       .orderBy(F.col("Count").desc())\
       .show(100,False)

+-------------------------------+-----+
|CallType                       |Count|
+-------------------------------+-----+
|Medical Incident               |7004 |
|Alarms                         |1144 |
|Structure Fire                 |906  |
|Traffic Collision              |433  |
|Outside Fire                   |153  |
|Other                          |114  |
|Citizen Assist / Service Call  |113  |
|Gas Leak (Natural and LP Gases)|69   |
|Water Rescue                   |43   |
|Elevator / Escalator Rescue    |36   |
|Electrical Hazard              |30   |
|Vehicle Fire                   |28   |
|Smoke Investigation (Outside)  |28   |
|Odor (Strange / Unknown)       |10   |
|Fuel Spill                     |10   |
|HazMat                         |5    |
|Train / Rail Incident          |5    |
|Suspicious Package             |3    |
|Assist Police                  |1    |
|Explosion                      |1    |
+-------------------------------+-----+



In [ ]:
#Задача 3 в какие месяцы 2018 года было больше всего вызовов пожарных
a = (fire_df.select(F.month("IncidentDate").alias('MONTH'), F.year('IncidentDate').alias('year'))
            .filter(F.year('IncidentDate') == 2018)
            .groupBy('MONTH', 'year')
            .agg(F.count('*').alias('count_calls'))
            .orderBy(F.col('count_calls').desc())
            .show(100,False))
a

+-----+----+-----------+
|MONTH|year|count_calls|
+-----+----+-----------+
|10   |2018|1068       |
|5    |2018|1047       |
|3    |2018|1029       |
|8    |2018|1021       |
|1    |2018|1007       |
|7    |2018|974        |
|6    |2018|974        |
|9    |2018|951        |
|4    |2018|947        |
|2    |2018|919        |
|11   |2018|199        |
+-----+----+-----------+



In [ ]:
#Задача 4 в каком районе Сан Франциско произошло больше всего звонков о пожаре в 2018?
a = (fire_df
           .select('Neighborhood', 'NumAlarms')
           .where((F.col('City') == 'San Francisco') & (F.year('IncidentDate') == 2018))
           .groupBy('Neighborhood')
           .agg(F.sum('NumAlarms').alias('Count_calls'))
           .orderBy(F.col('Count_calls').desc())
           .show(1, False))
a

+------------+-----------+
|Neighborhood|Count_calls|
+------------+-----------+
|Tenderloin  |1393       |
+------------+-----------+
only showing top 1 row



In [ ]:
#Задание 5 в каких районах было худшее время реагирования пожарных в 2018 году?
a = (fire_df
           .select('City', 'Neighborhood', 'Delay')
           .filter(F.year('IncidentDate') == 2018)
           .orderBy(F.col('Delay').desc())
           .show(5, False))

+-------------+------------------------------+---------+
|City         |Neighborhood                  |Delay    |
+-------------+------------------------------+---------+
|San Francisco|Chinatown                     |491.26666|
|San Francisco|Financial District/South Beach|406.63333|
|San Francisco|Tenderloin                    |340.48334|
|San Francisco|Haight Ashbury                |175.86667|
|San Francisco|Bayview Hunters Point         |155.8    |
+-------------+------------------------------+---------+
only showing top 5 rows



In [ ]:
#Задание 6 на какой неделе 2018 года было больше всего вызовов пожарных?
a = (fire_df
           .select('IncidentDate', 'NumAlarms')
           .filter(F.year('IncidentDate') == 2018)
           .groupBy(F.weekofyear('IncidentDate'))
           .agg(F.sum('NumAlarms').alias('sums'))
           .orderBy(F.col('sums').desc())
           .show(5, False))

+------------------------+----+
|weekofyear(IncidentDate)|sums|
+------------------------+----+
|22                      |259 |
|40                      |255 |
|43                      |252 |
|25                      |250 |
|1                       |246 |
+------------------------+----+
only showing top 5 rows



In [ ]:
#Задание 7 как использовать тип файлов Паркет, для хранения этих данных.
fire_df.write.format('parquet').save('/content/sample_data/qwe')